In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from data_processing.start_experiment import *
from data_processing.read_data import *
from data_processing.gaussian import *
import numpy as np

In [9]:
parameter_list = ['season_duration', 'gamma', 'mut_rate', 'T', 'target_area', 'gradnoise']

In [10]:
parameter_space = ParameterSpace([DiscreteParameter('season_duration',list(range(100,1001,10))), 
                                 DiscreteParameter('gamma',list(range(0,25))), 
                                 ContinuousParameter('mut_rate',0,0.5),
                                 ContinuousParameter('T',1,100),
                                 DiscreteParameter('target_area',list(range(1,101))),
                                 ContinuousParameter('gradnoise',0.25,0.99)])

In [11]:
num_experiments = 100

In [12]:
file_names = ["latin/latin_{}.par".format(i) for i in range(num_experiments)]
data_files = ["latin/data_cellcount_{}.txt".format(i) for i in range(num_experiments)]

In [13]:
X = get_parameter_array(file_names,parameter_list)

# Some files have no data; don't get those
unrun_files = get_no_data(data_files) 
X_no_data = X[unrun_files,:]

X = np.delete(X,unrun_files,axis=0)

In [14]:
X.shape

(87, 6)

In [15]:
Y = get_rewards(data_files,average_function_over_time(average_distance_between_cells))

In [16]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)
model_emukit.optimize()

In [23]:
print(X)

[[5.6000e+02 1.8000e+01 4.7500e-02 8.4655e+01 1.5000e+01 4.6090e-01]
 [3.8000e+02 4.0000e+00 1.1250e-01 3.6145e+01 3.4000e+01 8.4570e-01]
 [2.6000e+02 1.2000e+01 3.2750e-01 9.0595e+01 8.1000e+01 5.4970e-01]
 [1.2000e+02 1.0000e+00 4.7250e-01 3.5155e+01 2.4000e+01 8.5310e-01]
 [7.7000e+02 7.0000e+00 3.7500e-02 8.0695e+01 6.4000e+01 7.6430e-01]
 [1.7000e+02 6.0000e+00 1.7750e-01 4.6045e+01 9.9000e+01 4.4610e-01]
 [9.3000e+02 1.1000e+01 4.2500e-02 5.9905e+01 5.8000e+01 9.4190e-01]
 [3.2000e+02 1.3000e+01 1.8250e-01 1.4950e+00 5.2000e+01 5.7930e-01]
 [5.4000e+02 2.0000e+00 3.5250e-01 9.2575e+01 8.8000e+01 6.5330e-01]
 [1.9000e+02 1.3000e+01 4.3250e-01 3.2185e+01 5.7000e+01 6.7550e-01]
 [7.3000e+02 2.1000e+01 2.0750e-01 7.7725e+01 3.7000e+01 5.6450e-01]
 [7.3000e+02 1.3000e+01 3.6250e-01 1.1395e+01 2.9000e+01 8.2350e-01]
 [4.6000e+02 2.2000e+01 4.5750e-01 4.5055e+01 2.8000e+01 5.4230e-01]
 [3.0000e+02 1.7000e+01 3.5750e-01 8.4250e+00 4.7000e+01 6.3110e-01]
 [8.4000e+02 1.2000e+01 2.5000e-03

In [17]:
senstivity_ishigami_gpbased = MonteCarloSensitivity(model = model_emukit, input_domain = parameter_space)
main_effects_gp, total_effects_gp, _ = senstivity_ishigami_gpbased.compute_effects(num_monte_carlo_points = 10000)

In [20]:
main_effects_gp

{'season_duration': array([-0.00060495]),
 'gamma': array([-0.00060495]),
 'mut_rate': array([-0.00060495]),
 'T': array([-0.00060495]),
 'target_area': array([-0.00060495]),
 'gradnoise': array([-0.00060495])}

In [21]:
total_effects_gp

{'season_duration': array([1.00060495]),
 'gamma': array([0.99548831]),
 'mut_rate': array([0.90731089]),
 'T': array([1.0006043]),
 'target_area': array([0.99850461]),
 'gradnoise': array([0.90677508])}